In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import tensorflow_addons as tfa
import os
from PIL import Image
import pandas as pd

import ecoset
import categorization
import utils

2022-06-23 22:05:27.518417: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Load model
ecosetModel = ecoset.make_alex_net_v2(
    weights_path='./models/AlexNet/ecoset_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89',
)
imagenetModel = ecoset.make_alex_net_v2(
    output_shape=1000, 
    weights_path='./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89',
)
ecosetModel.summary()

2022-06-23 22:05:28.627952: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-06-23 22:05:28.657108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 22:05:28.657440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 970 computeCapability: 5.2
coreClock: 1.253GHz coreCount: 13 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2022-06-23 22:05:28.657459: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-06-23 22:05:28.661096: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-06-23 22:05:28.661142: I tensorflow/stream_executor/plat

Weights from ./models/AlexNet/ecoset_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89 loaded successfully.
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89 loaded successfully.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 54, 54, 64)        23296     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 26, 26, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 26, 26, 192)       307392    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 12, 12, 192)       0         
_______________________

In [3]:
# model = ecoset.make_vNet(
#     weights_path='./models/vNET/ecoset_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch79',
#     softmax=True,)
# model = Imagenet.make_vNet(
#     output_shape=1000, 
#     weights_path='./models/vNET/ILSVRC_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch79',
#     softmax=True,
# )
# model.summary()

In [4]:
trainCats, trainCounts = categorization.build_categories_from_ecoset(
    './images/train', 
    includeSub=False,
    maxImgs=100
    )
testCats, testCounts = categorization.build_categories_from_ecoset(
    './images/val+test',
    includeSub=False,
    maxImgs=100
)

trainImages = categorization.get_images_from_cat(trainCats, preprocFun=ecoset.preprocess_alexnet)
testImages = categorization.get_images_from_cat(testCats, preprocFun=ecoset.preprocess_alexnet)

In [5]:
ecosetModel = utils.make_output_model(ecosetModel)
imagenetModel = utils.make_output_model(imagenetModel)

In [42]:
birdTrainReps = ecosetModel.predict(trainImages['animal']['0085_bird'])[-3]
dogTrainReps = ecosetModel.predict(trainImages['animal']['0039_dog'])[-3]
carTrainReps = ecosetModel.predict(trainImages['vehicle']['0009_car'])[-3]
busTrainReps = ecosetModel.predict(trainImages['vehicle']['0089_bus'])[-3]

birdTestReps = ecosetModel.predict(testImages['animal']['0085_bird'])[-3]
dogTestReps = ecosetModel.predict(testImages['animal']['0039_dog'])[-3]
carTestReps = ecosetModel.predict(testImages['vehicle']['0009_car'])[-3]
busTestReps = ecosetModel.predict(testImages['vehicle']['0089_bus'])[-3]

In [43]:
# Perform categorization comparing against only the most similar category
withinCatEcosetData = pd.DataFrame(columns=['level', 'cat', 'resp', 'rt'])

for rep in birdTestReps:
    probs = categorization.sim_prob(rep, birdTrainReps, dogTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'
    
    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'basic',
        'cat': 'bird',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in dogTestReps:
    probs = categorization.sim_prob(rep, dogTrainReps, birdTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'basic',
        'cat': 'dog',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in carTestReps:
    probs = categorization.sim_prob(rep, carTrainReps, busTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'basic',
        'cat': 'car',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in busTestReps:
    probs = categorization.sim_prob(rep, busTrainReps, carTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'basic',
        'cat': 'bus',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((birdTestReps, dogTestReps)):
    probs = categorization.sim_prob(rep, [birdTrainReps, dogTrainReps], [carTrainReps, busTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'super',
        'cat': 'animal',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((carTrainReps, busTrainReps)):
    probs = categorization.sim_prob(rep, [carTrainReps, busTrainReps], [birdTrainReps, dogTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatEcosetData = withinCatEcosetData.append({
        'level': 'super',
        'cat': 'vehicle',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

In [44]:
# Group data basic on level and count responses
withinCatEcosetData.groupby(['level'])['resp'].value_counts()

level  resp
basic  yes     344
       no       56
super  yes     397
       no        3
Name: resp, dtype: int64

In [45]:
# Group data basic on level and get mean RTs
withinCatEcosetData.groupby(['level'])['rt'].mean()

level
basic    1.934697
super    1.871480
Name: rt, dtype: float64

In [46]:
birdTrainReps = imagenetModel.predict(trainImages['animal']['0085_bird'])[-3]
dogTrainReps = imagenetModel.predict(trainImages['animal']['0039_dog'])[-3]
carTrainReps = imagenetModel.predict(trainImages['vehicle']['0009_car'])[-3]
busTrainReps = imagenetModel.predict(trainImages['vehicle']['0089_bus'])[-3]

birdTestReps = imagenetModel.predict(testImages['animal']['0085_bird'])[-3]
dogTestReps = imagenetModel.predict(testImages['animal']['0039_dog'])[-3]
carTestReps = imagenetModel.predict(testImages['vehicle']['0009_car'])[-3]
busTestReps = imagenetModel.predict(testImages['vehicle']['0089_bus'])[-3]

In [47]:
# Perform categorization comparing against only the most similar category
withinCatImagenetData = pd.DataFrame(columns=['level', 'cat', 'resp', 'rt'])

for rep in birdTestReps:
    probs = categorization.sim_prob(rep, birdTrainReps, dogTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'
    
    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'basic',
        'cat': 'bird',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in dogTestReps:
    probs = categorization.sim_prob(rep, dogTrainReps, birdTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'basic',
        'cat': 'dog',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in carTestReps:
    probs = categorization.sim_prob(rep, carTrainReps, busTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'basic',
        'cat': 'car',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in busTestReps:
    probs = categorization.sim_prob(rep, busTrainReps, carTrainReps)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'basic',
        'cat': 'bus',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((birdTestReps, dogTestReps)):
    probs = categorization.sim_prob(rep, [birdTrainReps, dogTrainReps], [carTrainReps, busTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'super',
        'cat': 'animal',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((carTrainReps, busTrainReps)):
    probs = categorization.sim_prob(rep, [carTrainReps, busTrainReps], [birdTrainReps, dogTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    withinCatImagenetData = withinCatImagenetData.append({
        'level': 'super',
        'cat': 'vehicle',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

In [48]:
# Group data basic on level and count responses
withinCatImagenetData.groupby(['level'])['resp'].value_counts()

level  resp
basic  yes     354
       no       46
super  yes     357
       no       43
Name: resp, dtype: int64

In [49]:
# Group data basic on level and get mean RTs
withinCatImagenetData.groupby(['level'])['rt'].mean()

level
basic    1.923832
super    1.889256
Name: rt, dtype: float64

In [50]:
birdTrainReps = ecosetModel.predict(trainImages['animal']['0085_bird'])[-3]
dogTrainReps = ecosetModel.predict(trainImages['animal']['0039_dog'])[-3]
carTrainReps = ecosetModel.predict(trainImages['vehicle']['0009_car'])[-3]
busTrainReps = ecosetModel.predict(trainImages['vehicle']['0089_bus'])[-3]

birdTestReps = ecosetModel.predict(testImages['animal']['0085_bird'])[-3]
dogTestReps = ecosetModel.predict(testImages['animal']['0039_dog'])[-3]
carTestReps = ecosetModel.predict(testImages['vehicle']['0009_car'])[-3]
busTestReps = ecosetModel.predict(testImages['vehicle']['0089_bus'])[-3]

In [51]:
# Perform categorization comparing against all other categories
betweenCatEcosetData = pd.DataFrame(columns=['level', 'cat', 'resp', 'rt'])

for rep in birdTestReps:
    probs = categorization.sim_prob(rep, birdTrainReps, [dogTrainReps, carTrainReps, busTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'
    
    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'basic',
        'cat': 'bird',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in dogTestReps:
    probs = categorization.sim_prob(rep, dogTrainReps, [birdTrainReps, carTrainReps, busTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'basic',
        'cat': 'dog',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in carTestReps:
    probs = categorization.sim_prob(rep, carTrainReps, [busTrainReps, dogTrainReps, birdTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'basic',
        'cat': 'car',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in busTestReps:
    probs = categorization.sim_prob(rep, busTrainReps, [carTrainReps, dogTrainReps, birdTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'basic',
        'cat': 'bus',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((birdTestReps, dogTestReps)):
    probs = categorization.sim_prob(rep, [birdTrainReps, dogTrainReps], [carTrainReps, busTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'super',
        'cat': 'animal',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((carTrainReps, busTrainReps)):
    probs = categorization.sim_prob(rep, [carTrainReps, busTrainReps], [birdTrainReps, dogTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatEcosetData = betweenCatEcosetData.append({
        'level': 'super',
        'cat': 'vehicle',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

In [52]:
# Group data basic on level and count responses
betweenCatEcosetData.groupby(['level'])['resp'].value_counts()

level  resp
basic  yes     395
       no        5
super  yes     397
       no        3
Name: resp, dtype: int64

In [53]:
# Group data basic on level and get mean RTs
betweenCatEcosetData.groupby(['level'])['rt'].mean()

level
basic    1.881294
super    1.871480
Name: rt, dtype: float64

In [54]:
birdTrainReps = imagenetModel.predict(trainImages['animal']['0085_bird'])[-3]
dogTrainReps = imagenetModel.predict(trainImages['animal']['0039_dog'])[-3]
carTrainReps = imagenetModel.predict(trainImages['vehicle']['0009_car'])[-3]
busTrainReps = imagenetModel.predict(trainImages['vehicle']['0089_bus'])[-3]

birdTestReps = imagenetModel.predict(testImages['animal']['0085_bird'])[-3]
dogTestReps = imagenetModel.predict(testImages['animal']['0039_dog'])[-3]
carTestReps = imagenetModel.predict(testImages['vehicle']['0009_car'])[-3]
busTestReps = imagenetModel.predict(testImages['vehicle']['0089_bus'])[-3]

In [55]:
# Perform categorization comparing against all other categories
betweenCatImagenetData = pd.DataFrame(columns=['level', 'cat', 'resp', 'rt'])

for rep in birdTestReps:
    probs = categorization.sim_prob(rep, birdTrainReps, [dogTrainReps, carTrainReps, busTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'
    
    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'basic',
        'cat': 'bird',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in dogTestReps:
    probs = categorization.sim_prob(rep, dogTrainReps, [birdTrainReps, carTrainReps, busTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'basic',
        'cat': 'dog',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in carTestReps:
    probs = categorization.sim_prob(rep, carTrainReps, [busTrainReps, dogTrainReps, birdTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'basic',
        'cat': 'car',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in busTestReps:
    probs = categorization.sim_prob(rep, busTrainReps, [carTrainReps, dogTrainReps, birdTrainReps], equalize=True)
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'basic',
        'cat': 'bus',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((birdTestReps, dogTestReps)):
    probs = categorization.sim_prob(rep, [birdTrainReps, dogTrainReps], [carTrainReps, busTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'super',
        'cat': 'animal',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

for rep in np.concatenate((carTrainReps, busTrainReps)):
    probs = categorization.sim_prob(rep, [carTrainReps, busTrainReps], [birdTrainReps, dogTrainReps])
    resp, rt = categorization.LBA_deterministic(probs[0], probs[1])
    resp = 'yes' if resp == 1 else 'no'

    # Add row
    betweenCatImagenetData = betweenCatImagenetData.append({
        'level': 'super',
        'cat': 'vehicle',
        'resp': resp,
        'rt': rt
    }, ignore_index=True)

In [56]:
# Group data basic on level and count responses
betweenCatImagenetData.groupby(['level'])['resp'].value_counts()

level  resp
basic  yes     377
       no       23
super  yes     357
       no       43
Name: resp, dtype: int64

In [57]:
# Group data basic on level and get mean RTs
betweenCatEcosetData.groupby(['level'])['rt'].mean()

level
basic    1.881294
super    1.871480
Name: rt, dtype: float64